In [1]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] ="-1"
import torch
import numpy as np
import torch.nn as nn
# import torch.nn.functional as F
# import torchvision

#from keras.preprocessing.sequence import pad_sequences
#from keras.utils import to_categorical

from tqdm import tqdm_notebook as tqdm

import json
import nltk
from nltk import word_tokenize

from torch.utils.data.dataloader import default_collate


In [2]:
modelname = "models/Model123"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
DIC_word_index = json.load(open("vocab.json", "r", encoding='utf-8'))
DIC_index_word = {index:word for word, index in DIC_word_index.items()}
word_vectors = np.load("wv_matrix100d.npy")

cuda


In [3]:
# if you don't have money, use this cell

# device = torch.device("cpu")
# print(device)
# DIC_word_index = json.load(open("vocab.json", "r", encoding='utf-8'))
# DIC_index_word = {index:word for word, index in DIC_word_index.items()}
# word_vectors = np.load("wv_matrix100d.npy")

In [4]:
sent_len = 10

In [5]:
class InferenceAttModel(torch.nn.Module):
    def __init__(self, latent_dim, seq_length, embed_dim, word_vectors):
        super(InferenceAttModel, self).__init__()
        
        self.hidden_dim = latent_dim        
        self.seq_length = seq_length
        self.embed_dim = embed_dim
        
#         self.embedding = torch.nn.Embedding(len(word_vectors), embed_dim)
#         self.embedding.load_state_dict({'weight': word_vectors})
#         self.embedding.weight.requires_grad = False
        self.embedding = torch.nn.Embedding.from_pretrained(torch.from_numpy(word_vectors), freeze=True)
#        self.embedding = torch.nn.Embedding(len(word_vectors), embed_dim)
    
        self.soft = torch.nn.Softmax(-2)
        
        self.RNN1 = torch.nn.LSTM(self.embed_dim, latent_dim,  num_layers=2, batch_first = True)
        self.RNN2 = torch.nn.LSTM(self.embed_dim, latent_dim,  num_layers=2, batch_first = True)
        
        self.Projection_layer = torch.nn.Sequential(
            torch.nn.Linear(2 * latent_dim, len(word_vectors)),
        )
        
        self.trainable_W = torch.nn.Sequential(
            torch.nn.Linear(latent_dim ,  latent_dim),
            torch.nn.Sigmoid(),
            torch.nn.Dropout(p=0.2),
        )
    def encoder_RNN(self, e_x):
        e_y, hiddens = self.RNN1(e_x)
        
        return e_y, hiddens
    
    def decoder_RNN(self, d_x, hiddens):
        d_y, hiddens2 = self.RNN2(d_x, hiddens)
        return d_y, hiddens2
    
    def forward(self, e_x, ):
        e_x = self.embedding(e_x).float()
        batch_size = e_x.shape[0]
        """
        if(batch_size != 1):
            print(batch_size)
            print("Batch size > 1 is not available now. Batch size please give 1.")
            return
        """
        #print(sent.shape) torch.Size([2, 15, 6087])

        e_output, hiddens = self.encoder_RNN(e_x)
        #attention
        uW = self.trainable_W(e_output)
        
        first = True

        first_index = DIC_word_index['<bos>']
        first_input = torch.tensor([[first_index] * batch_size]).to(device)
        first_input = first_input.view(batch_size, 1, 1)
        for i in range(self.seq_length):
            #embed_dim default:100
            #seq_length = sent_len, default:5
            if(first):
                word = first_input;

                
            #word.shape: [batch_size, 1, 1]
            word = self.embedding(word.view(1, -1)).float();
            #word.shape: [1, batch_size, embed_dim]
            #one for each word, therefore d_output = d_state
            
            
            d_output, hiddens = self.decoder_RNN(word.transpose(1,0), hiddens)
            #d_output.shape: [batch_size, 1, hidden_dim]
            uWv = torch.matmul(uW, d_output.transpose(2,1))
            #uWv.shape: [batch_size, seq_length, 1]
            alpha = self.soft(uWv)

            new_context = e_output.transpose(2,1) @ alpha;
            #e_output.shape [batch_size, seq_length, hidden_dim]
            #alpha.shape [batch_size, seq_length, 1]
            
            #new_context.shape [batch_size, hidden_dim, 1]          
            pre = torch.cat((d_output, new_context.transpose(2,1)), 2)
            ans_logits = self.Projection_layer(pre)
            #ans_index = np.argmax(ans_logits.cpu().detach().numpy())
            
            values, indices = torch.max(ans_logits, -1)
            word = indices;
            #word.shape [batch_size, 1]
            if(first):
                batch_seq = word;
                first = False
            else:
                batch_seq = torch.cat((batch_seq, word), 1)
            
            #ans = DIC_index_word[ans_index]
            #ans_seq.append(ans)
            #word = torch.tensor([ans_index])
        
        #logits = torch.stack(logits, 1)
        return batch_seq

In [6]:
#
def Inference(model_path, question_path):
    def pad_post_zero(a, length):
        ret = []
        for _list in a:
            if(len(_list) < length):
                for ct in range(len(_list),length,1):
                    _list.append(DIC_word_index["<pad>"])
            if(len(_list) > length):
                _list = _list[:length]

            ret.append(_list)
        return ret
    def my_collate(datas):
        batch = {}
        batch['encoder_x'] = torch.tensor([data for data in datas])
        return batch
    latent_dim = 1024
    embed_dim = word_vectors.shape[1]
    batch_size = 64
    
    model = InferenceAttModel(latent_dim, sent_len, embed_dim, word_vectors)
    #saved_model = torch.load("models/Model92")
    saved_model = torch.load(model_path)
    model.load_state_dict(saved_model['model'])
    model = model.to(device)
    model.eval()
    
    encode_x = []
    #with open('sel_conversation/question.txt', 'r') as f:
    with open(question_path, 'r') as f:
        for l in f:
            l = l.split()
            tmp = []
            for word in l:
                try:
                    index = DIC_word_index[word]
                except KeyError:
                    index = DIC_word_index['<unk>']
                tmp.append(index)
            encode_x.append([DIC_word_index['<bos>']] + tmp + [DIC_word_index['<eos>']])        
    encode_x = pad_post_zero(encode_x, sent_len)
    
    #print(np.array(encode_x).shape); (len(encode_x, sent_len)
    dataloader = torch.utils.data.DataLoader(encode_x, batch_size = batch_size, shuffle = False, collate_fn = my_collate)
    ANSWERS = list()
    for i, batch in tqdm(enumerate(dataloader), total=len(dataloader)):
        with torch.no_grad():
            e_x = batch['encoder_x'].long().to(device)
        batch_seq = model.forward(e_x)
        for seq in batch_seq:
            #seq shape: [sent_len]
            ans = []
            for ans_index in seq:
                ans.append(DIC_index_word[ans_index.item()])
            ANSWERS.append(ans)
    return(ANSWERS)
    """
    ANSWERS = list()
    for index in range(5):
        e_x = torch.tensor([encode_x[index]]).long().to(device)
        ans = model.forward(e_x)
        ANSWERS.append(ans)
    
    return(ANSWERS)
    """

In [7]:
ANSWERS = Inference(model_path=modelname, question_path='evaluation/test_input.txt')

In [8]:
ANSWERS 

[['我們', '選', '了', '一個', '很', '特別', '的', '日子', '<eos>', '<eos>'],
 ['來', '紀念', '退伍軍人', '節', '<eos>', '<eos>', '想', '<eos>', '<eos>', '<eos>'],
 ['就是', '保護', '我們', '的', '軍隊', '<eos>', '責任', '<eos>', '<eos>', '<eos>'],
 ['保護', '為', '國家', '效力', '的', '男女', '<eos>', '<eos>', '<eos>', '國家'],
 ['我們',
  '的',
  '軍隊',
  '<unk>',
  '<eos>',
  '<eos>',
  '<eos>',
  '<eos>',
  '<eos>',
  '<eos>'],
 ['現在', '該', '讓', '他們', '回來', '了', '<eos>', '<eos>', '<eos>', '<eos>'],
 ['在', '今天', '這個', '節日', ',', '我們', '要', '對', '的', '<eos>'],
 ['他', '從', '這裡', '帶回', '<eos>', '這裡', '<eos>', '做法', '<eos>', '<eos>'],
 ['打到', '他', '了', '嗎', '?', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>'],
 ['我們', '<unk>', '通報', '後', '有', '一', '小時', '空檔', '<eos>', '<eos>'],
 ['我', '派', '了', '狙擊手', '和', '直升機', '出動', '<eos>', '<eos>', '警察'],
 ['對方', '就', '開槍', '了', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>'],
 ['在', '那裡', '和', '那裡', '那裡', '<eos>', '那裡', '<eos>', '<eos>', '<eos>'],
 ['你', '要', '爲', '他們', '的', '作業', '?', '<eos>

In [ ]:
with open("result.txt", "w") as f:
    for res in ANSWERS:
        try:
            end = res.index('<eos>')
        except ValueError:
            end = len(res)
        sent = "".join(res[:end])
        f.write(sent+'\n')

In [ ]:
def pad_post_zero(a, length):
    ret = []
    for _list in a:
        if(len(_list) < length):
            for ct in range(len(_list),length,1):
                _list.append(DIC_word_index["<pad>"])
        if(len(_list) > length):
            _list = _list[:length]

        ret.append(_list)
    return ret
def my_collate(datas):
    batch = {}
    batch['encoder_x'] = torch.tensor([data for data in datas])
    return batch

latent_dim = 1024
embed_dim = word_vectors.shape[1]
batch_size = 1

model = InferenceAttModel(latent_dim, sent_len, embed_dim, word_vectors)
saved_model = torch.load(modelname)
# saved_model = torch.load(model_path)
model.load_state_dict(saved_model['model'])
model = model.to(device)
model.eval()

InferenceAttModel(
  (embedding): Embedding(46801, 100)
  (soft): Softmax()
  (RNN1): LSTM(100, 1024, num_layers=2, batch_first=True)
  (RNN2): LSTM(100, 1024, num_layers=2, batch_first=True)
  (Projection_layer): Sequential(
    (0): Linear(in_features=2048, out_features=46801, bias=True)
  )
  (trainable_W): Sequential(
    (0): Linear(in_features=1024, out_features=1024, bias=True)
    (1): Sigmoid()
    (2): Dropout(p=0.2)
  )
)

In [ ]:
import jieba.posseg as pseg
while True:
    text = input()
    tmp = []
    for word, flag in pseg.cut(text):
        try:
            index = DIC_word_index[word]
        except KeyError:
            index = DIC_word_index['<unk>']
        tmp.append(index)      
    encode_x = pad_post_zero([[DIC_word_index['<bos>']] + tmp + [DIC_word_index['<eos>']]], sent_len)

    dataloader = torch.utils.data.DataLoader(encode_x, batch_size = batch_size, shuffle = False, collate_fn = my_collate)
    ANSWERS = []
    for batch in dataloader:
        with torch.no_grad():
            e_x = batch['encoder_x'].long().to(device)
        batch_seq = model.forward(e_x)
        for seq in batch_seq:
            #seq shape: [sent_len]
            ans = ""
            for ans_index in seq:
                w = DIC_index_word[ans_index.item()]
                if w != '<eos>':
                    ans += w
            ANSWERS.append(ans)
    print(ANSWERS)

早安


Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.643 seconds.
Prefix dict has been built succesfully.


['你昨晚了??']
